# Model the Data
Given a set of scalar data, you can model it with a suitable pdf. This exercise will help you learn the following:
- How would you choose the best model to fit the given data?
- How would you estimate the model parameters from the given data?
- Given a model, how do you sample new data from it?

Note: You are allowed to use only numpy and matplotlib libraries. No ML library.

In [8]:
import numpy as np
np.random.seed(111)

In [9]:
import copy

In [10]:
def data2gaussian(S):
    '''
    Return optimal parameters - (mu,sigma)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        sigma: float
    '''

    ### WRITE YOUR CODE HERE - 5 MARKS

    mu = np.sum(S)/len(S)
    sigma = 0
    for x in S:
      sigma = sigma + (x - mu)**2
    sigma = sigma/len(S)
    sigma = np.sqrt(sigma)

    return mu, sigma

In [11]:
def test_data2gaussian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, sigma = data2gaussian(S)
    print(mu,sigma)
    assert isinstance(mu, (int, float))
    assert isinstance(sigma, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2gaussian()

0.06000000000000001 0.19595917942265426
Test passed 👍


In [12]:
def data2laplacian(S):
    '''
    Return optimal parameters - (mu,b). See https://en.wikipedia.org/wiki/Laplace_distribution
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        mu: float
        b: float
    '''
    mu = np.median(S)
    b = 0
    for x in S:
      b = b + abs(x - mu)
    b = b/len(S)

    ### WRITE YOUR CODE HERE - 5 MARKS

    return mu, b

In [13]:
def test_data2laplacian(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    mu, b = data2laplacian(S)
    print(mu,b)
    assert isinstance(mu, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2laplacian()

0.0 0.14
Test passed 👍


In [14]:
def data2uniform(S):
    '''
    Return optimal parameters - (a,b)
    Inputs:
        S: np array of shape (Ns,). These are samples of a random variable.
    Outputs:
        a: float
        b: float
    '''
    b = np.max(S)
    a = b - (np.max(S) - np.min(S))
    # a = b - 2*(np.sum(S))/len(S)

    ### WRITE YOUR CODE HERE - 5 MARKS

    return a, b

In [15]:
def test_data2uniform(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    a, b = data2uniform(S)
    print(a,b)
    assert isinstance(a, (int, float))
    assert isinstance(b, (int, float))
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2uniform()

-0.20000000000000007 0.4
Test passed 👍


In [16]:
def gaussian(x, mu, sigma):
  x = np.array(x)
  return np.exp(-1*(x - mu)*(x-mu)/(2*(sigma**2)))/(np.sqrt(2*np.pi)*sigma)

def laplacian(x,mu,b):
  x = np.array(x)
  return np.exp(-1*np.abs(x - mu)/b)/(2*b)

def uniform(x,a,b):
  u = []
  for i in x:
    if a<=i<=b:
      u.append(1/(b-a))
    else:
      u.append(0)
  return np.array(u)

def data2model(S):
    '''
    Inputs:
        S: np array of shape (Ns,). These are scalar samples of a random variable.
    Outputs:
        modelName: return one out of these - 'gaussian', 'laplacian', or 
                   'uniform' which best models the data
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS

    models = ['gaussian', 'laplacian', 'uniform']

    mu,sigma = data2gaussian(S)
    mu_l, b_l = data2laplacian(S)
    a,b = data2uniform(S)

    likelihood = []
    likelihood.append(np.sum(np.log10(gaussian(S, mu,sigma))))
    likelihood.append(np.sum(np.log10(laplacian(S, mu_l,b_l))))
    likelihood.append(np.sum(np.log10(uniform(S, a,b))))

    modelName = models[np.argmax(likelihood)]

    return modelName

In [17]:
def test_data2model(): # checks formatting only
    S = [0.1,-0.2,0.4,0,0]
    modelName = data2model(S)
    assert modelName in ['gaussian', 'laplacian', 'uniform']
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2model()

Test passed 👍


In [18]:
def sampleGMM(pi, mu, sigma, Ns=1):
    '''
    Inputs:
        pi: np.array of shape (K,), p(z_k)
        mu: np.array of shape (K,), mu of kth gaussian
        sigma: np.array of shape (K,), sigma of kth gaussian
        Ns: int, number of samples
    Outputs:
        S: np.array of shape (Ns,), samples from the GMM
    '''

    ### WRITE YOUR CODE HERE - 10 MARKS

    S = []
    for i in range(Ns):
      k = np.random.choice(np.arange(0,len(pi)), p=pi)
      x = np.random.normal(mu[k],sigma[k],1)
      S.append(x[0])
    
    S = np.array(S)




    return S

In [19]:
def test_sampleGMM(): # checks formatting only
    pi = [0.3,0.7]
    mu = [-1.1, 1.3]
    sigma = [1.5, 0.4]
    Ns = 5
    S = sampleGMM(pi, mu, sigma, Ns)
    assert S.shape==(5,)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_sampleGMM()

Test passed 👍


In [25]:
np.random.seed(0)

def multi_gaussian(x, mu, sigma):
  x = np.array(x)
  ans = -(0.5)* np.dot(np.dot((x - mu), np.linalg.inv(sigma)) , (x-mu).T)
  ans = np.exp(ans)
  k = mu.shape[0]
  det = np.linalg.det(sigma)
  if det<=0:
    det = 10**(-2)
  ans = ans/np.sqrt((2*np.pi)**(k)*det)

  return ans

def data2GMM(S, K):
    '''
    Return optimal parameters - (pi,mu,sigma)
    Inputs:
        S: np array of shape (Ns,Na). These are samples of a random variable. Na can be 1, 2 or 3
    Outputs:
        pi: np array of shape (K,)
        mu: np array of shape (K,Na)
        sigma: np array of shape (K,Na,Na)
    '''

    ### WRITE YOUR CODE HERE - 15 MARKS

    
    # initialization
    pi = np.array([1/K]*K)
    Ns = S.shape[0]
    dim = S.shape[1]
    sigma = np.random.random((K,dim,dim))
    mu = np.random.random((K,dim))

    r = np.zeros((Ns,dim))
    for i in range(Ns):
      S_i = np.expand_dims(S[i],axis = 0)
      s = 0
      for k in range(K):
        mu_k = np.expand_dims(mu[k],axis = 0)
        s = s + pi[k]*multi_gaussian(S_i, mu_k, sigma[k])
      for k in range(K):
        r[i][k] = pi[k]*multi_gaussian(S_i, mu_k, sigma[k])/s

    
    
    eps = 10**(-3)
    ct = 0


    while 1:
      ct+=1
      # print(ct)
      mu_old = copy.deepcopy(mu)
      sigma_old = copy.deepcopy(sigma)
      pi_old = copy.deepcopy(pi)

      #pi
      for i in range(K):
        c = 0
        for j in range(Ns):
          c = c + r[j][i]
        pi[i] = c/Ns

      #mu
      for i in range(K):
        x = np.zeros((dim))
        c = 0
        for j in range(Ns):
          c = c + r[j][i]
          x = x + r[j][i]*S[j]
        if c!=0:
          mu[i] = x/c
      
      #sigma
      for i in range(K):
        x = np.zeros((dim,dim))
        c = 0
        mu_k = np.expand_dims(mu[i],axis = 0)
        for j in range(Ns):
          S_j = np.expand_dims(S[j],axis = 0)
          c = c + r[j][i]
          x = x + r[j][i]*np.matmul((S_j - mu_k).T, (S_j - mu_k))
        if c!=0:
          sigma[i] = x/c
      
      # try:
      #   for k in range(K):
      #     inv = np.linalg.inv(sigma[k])
      # except:
      #   print(sigma[k])
      #   print("here")
      #   break
      
      #r
      for i in range(Ns):
        S_i = np.expand_dims(S[i],axis = 0)
        s = 0
        for k in range(K):
          mu_k = np.expand_dims(mu[k],axis = 0)
          s = s + pi[k]*multi_gaussian(S_i, mu_k, sigma[k])
        for k in range(K):
          r[i][k] = pi[k]*multi_gaussian(S_i, mu_k, sigma[k])/s

      #convergence
      change = np.linalg.norm(mu - mu_old) + np.linalg.norm(sigma - sigma_old) + np.linalg.norm(pi - pi_old)
      if change < eps:
        break

    return pi, mu, sigma

In [24]:
def test_data2GMM(): # checks formatting only
    np.random.seed(0)
    S = np.random.random((10,3))
    pi, mu, sigma = data2GMM(S,2)
    assert pi.shape==(2,)
    assert mu.shape==(2,3)
    assert sigma.shape==(2,3,3)
    print('Test passed', '\U0001F44D')
if __name__=="__main__":
    test_data2GMM()

Test passed 👍
